In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [3]:
mc_data = pd.read_json('DataSets/SciFi_200000_Clean_TrackStudy.json')
# real_data = pd.read_json('DataSets/SciFi_200000_AllHits.json')

In [4]:
###### Data columns
i = 0
for val in mc_data.columns:
    print(i," ",val)
    i = i + 1
# Data columns
# i = 0
# for val in real_data.columns:
#    print(i," ",val)
#    i = i + 1

0   MCParticleID
1   MC_Hit_X
2   MC_Hit_Y
3   MC_Hit_Z
4   PrHit_LHCbID
5   PrHit_Xat0
6   PrHit_Zat0
7   PrHit_isX
8   PrHit_planeCode
9   PrHit_w2
10   PrHit_yMax
11   PrHit_yMin
12   PrHit_zone
13   event


In [5]:
mc_hits = mc_data.values
# real_hits = real_data.values

In [6]:
# Create X and Y matrices for the future model
length = mc_hits.shape[0]

X = np.zeros((length, 4))
Y = np.zeros((length, 11))

i = 0
empty_particles = []
for i in range(0,length):
    
    if ((len(mc_hits[i][5]) > 1)):
        # Get the first element of the hit, in the first layer
        X[i][0] = mc_hits[i][5][0]
        X[i][1] = mc_hits[i][10][0]
        X[i][2] = mc_hits[i][11][0]
        X[i][3] = mc_hits[i][12][0]
        
        # Get only the first element if there are several repeated with small differences
        j = 0
        extra_hits = mc_hits[i][5]
        try:
            while (np.abs(extra_hits[j] - extra_hits[j + 1]) < 5):
                j += 1
            j += 1
        except IndexError as e:
            X[i] = np.zeros(4)
            continue
        
        # Get all hits from the given particle which are not located in the first layer
        extra_hits = mc_hits[i][5][j:]
        actual_pos = 0
        j = 0
        try:
            while (j < len(extra_hits)):
                Y[i][actual_pos] = extra_hits[j]
                # Same as above, get only the first element in case there are several hits with similar X position
                while (j + 1 < len(extra_hits) and np.abs(extra_hits[j] - extra_hits[j + 1]) < 3):
                    j += 1
                actual_pos += 1
                j += 1
        except IndexError:
            X[i] = np.zeros(4)
            Y[i] = np.zeros(11)
            
# Removing empty values, as some hits in the mc_hits dataset do not contain readings (PrHit...)
Y = Y[Y[:,0] != 0]
X = X[X[:,0] != 0]

In [10]:
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score, KFold
from keras.models import Sequential
from sklearn.metrics import accuracy_score
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

def baseline_model():
    model = Sequential()
    model.add(Dense(4, input_dim=4, activation='relu'))
    model.add(Dense(11))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=100, verbose=False)
kfold = KFold(n_splits=10, random_state=1)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

estimator.fit(X, Y)

Results: -263169.16 (168973.54) MSE


In [12]:

prediction = estimator.predict(X)
accuracy_score(Y, prediction)

ValueError: continuous-multioutput is not supported

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [13]:
def showPredictedTrack(X, Y):
    fig = plt.figure(figsize=(19, 8))
    ax = fig.add_subplot(111)
    ax.plot(X, Y)       
    ax.grid()
    plt.show()

In [18]:
model.predict(X[0])

array([[ -822.90564,  -848.7039 ,  -902.64825, -1045.0826 , -1073.9144 ,
        -1079.4749 , -1123.5577 , -1241.6849 , -1231.1355 , -1128.6902 ,
         -798.90826]], dtype=float32)

In [19]:
Y[0]

array([ -820.46807861,  -987.8550415 , -1013.53503418, -1529.06494141,
       -1561.84423828, -1734.04956055, -1768.51501465, -2326.09008789,
       -2365.92919922, -2536.2019043 , -2576.20996094])